In [1]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# load the data
data = pd.read_csv('../data/merged_data.csv')

data.shape

(1920, 434)

In [2]:
data[:5]

,patdeid,attendance_0,attendance_1,attendance_2,attendance_3,attendance_4,attendance_5,attendance_6,attendance_7,attendance_8,...,pex_head_neck,pex_cardio,pex_skin,sae_name,sae_cat_death,sae_cat_life_threat,sae_cat_hospitalization,sae_cat_disability,medication,attendance
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0,0.0,0.0,0.0,0.0,2.0,25
1,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,0,0.0,0.0,0.0,0.0,2.0,25
2,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,0,0.0,0.0,0.0,0.0,1.0,25
3,4,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0,0.0,0.0,0.0,0.0,2.0,25
4,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,9.0,9.0,0,0.0,0.0,0.0,0.0,2.0,1
